In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
headers = {
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36'
}

In [3]:
r = requests.get("https://www.timesjobs.com/", headers=headers)
soup = BeautifulSoup(r.content, 'lxml')

In [4]:
belong_to_it_li = soup.find('li', class_='top-ind-it wht-shd-bx')
href_out_a = str([a_href['href'].split("<li>") for a_href in belong_to_it_li.select('a', href = True)]).strip("[").strip("]").strip("'")

In [5]:
href_out_a

'https://www.timesjobs.com/candidate/job-search.html?searchType=Home_Search&from=submit&asKey=OFF&txtKeywords=&cboPresFuncArea=35'

In [ ]:
all_data_list = []
for x in range(1,800):
    if x <=1:
        all_href = href_out_a
    elif x>=2:
        all_href = (href_out_a+f"&pDate=Y&sequence=2&startPage={x}")
    else:
        print("Good!! You're good")
    get_IT_requ = requests.get(all_href, headers = headers)
    it_soup = BeautifulSoup(get_IT_requ.content, 'lxml')

    job_title = it_soup.find_all('li', class_='clearfix job-bx wht-shd-bx')

    for title in job_title:

        three_data = title.find('header', class_='clearfix')
        loc_experi = title.find('ul', class_='top-jd-dtl clearfix')
        des_skills = title.find('ul', class_='list-job-dtl clearfix')

        # Job title
        job_name=', '.join([joby.get_text() for joby in three_data.find_all('h2')]).strip()

        # Company Name
        company_name= ', '.join([com.get_text() for com in three_data.find_all('h3')]).strip()

        # Experience
        experience = loc_experi.find('li').find(text=True, recursive=False)
        if experience is None:
            experience = "Experience Not Mentioned"
        else:
            experience = experience

        # Location
        location = loc_experi.find('span').find(text=True, recursive=False)
        if location is None:
            location = "Location Not Found"
        else:
            location = location

        # Description
        des = des_skills.find('li')
        des.label.decompose()
        description = des.text.strip()

        # Key Skills
        key_skill = des_skills.find('span', class_='srp-skills').text.strip()

        # Job Detail Link
        job_detail_link='/ '.join([d_link['href']for d_link in three_data.find_all('a', href = True)]).strip()


        all_data_dict = {
            'Job Title': job_name,
            'Company Name': company_name,
            'Experience': experience,
            'Location': location,
            'Job Description': description,
            'Key Skills': key_skill,
            'Job Detail Link': job_detail_link
        }
        all_data_list.append(all_data_dict)
        df = pd.DataFrame(all_data_list)
df.index = df.index+1
df.to_excel('D:\Programming\Python Coding\DcodeTech Projects\Times Jobs\Data\Times_Jobs.xlsx', index_label= 'Sr No.')
